In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np
import os
import yaml
from easydict import EasyDict

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import normalized_mutual_info_score as NMI

from z_common import config_to_execute
from data_aug.gaussian_blur import GaussianBlur
from clustering import run_clustering
from dataset import get_datasets
from simclr import SimCLR
from log_functions import *

from models.baseline_encoder import Encoder
from models.alexnet_simclr import AlexSimCLR
from models.resnet_simclr import ResNetSimCLR
from loss.nt_xent import NTXentLoss

/home/syamagami/anaconda3/envs/generic/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Please install apex for mixed precision training from: https://github.com/NVIDIA/apex


In [3]:
N_CUDA = 0
PARENT = 'Office31'
DATA_DOMAIN = 'dslr_webcam'
log_dir_opt = f''

config, CONFIG_FILE, log_dir = config_to_execute(DATA_DOMAIN, PARENT, N_CUDA, log_dir_opt)

==============  dslr_webcam  ==============


In [4]:
config.config_path = CONFIG_FILE
config.log_dir = log_dir
config.lap = 1
# feats, edls_dataset = run_clustering(config)

dataset = get_datasets(config, 'train')
train_loader = torch.utils.data.DataLoader(dataset, batch_size=config.batch_size, shuffle=True, num_workers=2, drop_last=True)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  68 tasks      | elapsed:    2.3s
[Parallel(n_jobs=4)]: Done 498 out of 498 | elapsed:    6.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 795 out of 795 | elapsed:    2.8s finished


In [5]:


model = Encoder(config.model.ssl, input_dim=3, out_dim=config.model.out_dim).cuda()
nt_xent_criterion = NTXentLoss('cuda', config.batch_size, **config.loss)

xis, xjs = next(iter(train_loader))
xis, xjs = xis.cuda(), xjs.cuda()
# get the representations and the projections
ris, zis = model(xis)  # [N,C]
rjs, zjs = model(xjs)  # [N,C]
# normalize projection feature vectors
zis = F.normalize(zis, dim=1)
zjs = F.normalize(zjs, dim=1)

loss = nt_xent_criterion(zis, zjs)
loss

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      

tensor(5.2785, device='cuda:0', grad_fn=<DivBackward0>)